In [3]:
from lxml import html
import requests
import time

def is_header(row):
    if len(row.xpath('th')):
        return True
    else:
        return False

def decode_unicode(data):
    return [x.encode('utf-8') for x in data]
    
def scrape_rows(rows):
    numbeo_sections = []
    for row in rows:
        if is_header(row):
            header_data = decode_unicode(row.xpath('th/text()'))
            section = dict(
                title=header_data[0],
                data=[]
            )
            numbeo_sections.append(section)
        else:
            row_data = decode_unicode(row.xpath('td/text()'))
            expense = row_data[0]
            cost = row_data[1]
            data_row = dict(expense=expense, cost=cost)

            # take the previous
            section = numbeo_sections[-1]
            section['data'].append(data_row)
            
    
    return numbeo_sections

def scrape_countries():    
    page = requests.get('https://www.numbeo.com/cost-of-living')
    tree = html.fromstring(page.content)
    countries = tree.xpath('//select[@id="country"]/option/text()')[1:]

    return countries


countries = scrape_countries()

countries_data = dict()

for country in countries:
    page = requests.get('https://www.numbeo.com/cost-of-living/country_result.jsp?country={}&displayCurrency=USD'.format(country))
    tree = html.fromstring(page.content)
    rows = tree.xpath('//table[@class="data_wide_table"]/tr')
    country_data = scrape_rows(rows)
    countries_data[country] = country_data

In [4]:
countries_data['Canada']

[{'data': [{'cost': ' 11.96\xc2\xa0$',
    'expense': 'Meal, Inexpensive Restaurant '},
   {'cost': ' 51.04\xc2\xa0$',
    'expense': 'Meal for 2 People, Mid-range Restaurant, Three-course '},
   {'cost': ' 7.18\xc2\xa0$',
    'expense': 'McMeal at McDonalds (or Equivalent Combo Meal) '},
   {'cost': ' 4.78\xc2\xa0$', 'expense': 'Domestic Beer (0.5 liter draught) '},
   {'cost': ' 5.18\xc2\xa0$', 'expense': 'Imported Beer (0.33 liter bottle) '},
   {'cost': ' 3.05\xc2\xa0$', 'expense': 'Cappuccino (regular) '},
   {'cost': ' 1.58\xc2\xa0$', 'expense': 'Coke/Pepsi (0.33 liter bottle) '},
   {'cost': ' 1.35\xc2\xa0$', 'expense': 'Water (0.33 liter bottle)  '}],
  'title': 'Restaurants'},
 {'data': [{'cost': ' 1.65\xc2\xa0$', 'expense': 'Milk (regular), (1 liter) '},
   {'cost': ' 2.20\xc2\xa0$', 'expense': 'Loaf of Fresh White Bread (500g) '},
   {'cost': ' 3.13\xc2\xa0$', 'expense': 'Rice (white), (1kg) '},
   {'cost': ' 2.62\xc2\xa0$', 'expense': 'Eggs (regular) (12) '},
   {'cost': ' 

In [5]:
import json

numbeo_data_file = 'countries_data.json'

with open(numbeo_data_file, 'w') as fp:
    fp.write(json.dumps(countries_data))